In [1]:
import numpy as np

## algorithm

In [2]:
class HashTable:

    ratio_expand = .7
    ratio_shrink = .2
    min_size = 11
    empty = (None,)

    def __init__(self, size=None):
        self._size = size or self.min_size
        self._buckets = [None] * self._size
        self._count = 0

    def _entry(self, key):
        # get hash
        hash_ = hash(key)
        idx1 = None

        for i in range(self._size):
            # quadratic probing
            idx = (hash_ + i) % self._size
            entry = self._buckets[idx]

            # end of chain
            if not entry:
                break
            # remember first empty bucket
            elif entry is self.empty:
                if idx1 is None:
                    idx1 = idx
            # test key
            elif entry[0] == key:
                return idx, entry

        else:
            # out of space
            if idx1 is None:
                raise IndexError()

        # return first empty bucket
        return (idx, None) if idx1 is None else (idx1, None)

    def _ensure_capacity(self):
        fill = self._count / self._size
        
        # expand or shrink?
        if fill > self.ratio_expand:
            self._size = self._size * 2 + 1
        elif fill < self.ratio_shrink and self._size > self.min_size:
            self._size = (self._size - 1) // 2
        else:
            return

        # reallocate buckets
        entries = self._buckets
        self._buckets = [None] * self._size

        # store entries into new buckets
        for entry in entries:
            if entry and entry is not self.empty:
                idx, _ = self._entry(entry[0])
                self._buckets[idx] = entry

    def __len__(self):
        return self._count

    def __contains__(self, key):
        _, entry = self._entry(key)
        return bool(entry)

    def __getitem__(self, key):
        _, entry = self._entry(key)
        return entry and entry[1]

    def __setitem__(self, key, value):
        idx, entry = self._entry(key)

        # set value
        self._buckets[idx] = key, value

        # expand
        self._count += bool(not entry or entry is self.empty)
        self._ensure_capacity()

    def __delitem__(self, key):
        idx, entry = self._entry(key)

        # delete key and value
        if entry:
            self._buckets[idx] = self.empty

        # shrink
        self._count -= bool(entry and entry is not self.empty)
        self._ensure_capacity()

    def __iter__(self):
        for entry in self._buckets:
            if entry and entry is not self.empty:
                yield entry[0]

    def slots(self):
        return ''.join('-' if not p else 'o' if p is self.empty else 'x' for p in self._buckets)


## run

In [3]:
table = HashTable()

In [4]:
# add random values
for _ in range(1000):
    key, value = np.random.randint(1000), np.random.rand()
    if np.random.rand() >= .5:
        table[key] = value
    else:
        del table[key]

In [5]:
len(table), table._size

(303, 767)

In [6]:
table.slots()

'-xxxxxxxx----xxxx--xxxx--xxxxxxxxx--x-xx------x-xx-xx---xxxxxxxx--------xxx-----x-x-x---x--xxxxxxx--xxxxo-x-x----xx-xxx-xo-xxx----xx---xxxx--xx-x-----xxxxxxx-xxxx-xxx-x---x-x----o---xx-----xx---xxxxx---xx-xxxxxo--xx----xxxxxxx--x--x-x-xxx-----x--------x-x---x--xx--------xx-x-------x--x-xx-x---x-x--xx-x--------x--x-x-xxx--x-----xx-----xx-xo------x-x--x-x----x-------xxx-x-x----x--x-x-----xx----xxxx----x-xx--------x---xx--x--x--x--xx--xx---x--x---x-----x--x--xx---x-x-x--x-x-ox--x-xx-x---xx--xox-xo-----x-x---xx-x---x-x----x--x-x-----x--xxx----x-----ox--xx-x--x-xx-x-xx-x---xx--xxo-ox--xx-x-x---xx-x--x-x---xx-x-ox-xxxx--x-----------x-------x-xx-xxx---xxx--x-----------xx-x-----x-------x---x------x--xxxx-------x-x---x--x-xxx---o-xx----o------x--x---xx-x---xx-xxx-x-'

In [7]:
# print some values
for key in list(table)[:5]:
    print(key, table[key])

1 0.09786699342385574
2 0.3816624750531391
768 0.17267164120539713
769 0.5484495883897172
772 0.2314199080141074


In [8]:
# delete all the values
for key in list(table):
    del table[key]

In [9]:
len(table), table._size

(0, 11)